In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset = pd.get_dummies(dataset,drop_first = True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [11]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [15]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [17]:
indep = dataset[['User ID', 'Age', 'EstimatedSalary', 'Gender_Male']]
dep = dataset['Purchased']

In [19]:
indep.shape

(400, 4)

In [21]:
indep

,User ID,Age,EstimatedSalary,Gender_Male
0,15624510,19,19000,True
1,15810944,35,20000,True
2,15668575,26,43000,False
3,15603246,27,57000,False
4,15804002,19,76000,True
...,...,...,...,...
395,15691863,46,41000,False
396,15706071,51,23000,True
397,15654296,50,20000,False
398,15755018,36,33000,True


In [23]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(indep,dep, test_size = 0.30, random_state = 0)

In [27]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [31]:
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV

param_grid = {'kernel':['linear','rbf','poly','sigmoid'],
             'gamma':['auto','scale'],
             'C':[10,100,1000,2000,3000]} 



grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3,cv=5,n_jobs=-1,scoring='f1_weighted') 

grid.fit(X_train, Y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(cv=5, estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [35]:
result=grid.cv_results_
#print(result)

Y_pred = grid.predict(X_test)
#print(y_pred)
   
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Y_test, Y_pred)
#print(cm)

from sklearn.metrics import classification_report

clf_report = classification_report(Y_test, Y_pred)

#print(clf_report)

In [37]:
from sklearn.metrics import f1_score

f1_macro=f1_score(Y_test,Y_pred,average='weighted')

print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}: 0.8906190214115365


In [39]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[74  5]
 [ 8 33]]


In [41]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.90      0.94      0.92        79
           1       0.87      0.80      0.84        41

    accuracy                           0.89       120
   macro avg       0.89      0.87      0.88       120
weighted avg       0.89      0.89      0.89       120



In [45]:
#AUC ROC stands for “Area Under the Curve” of the “Receiver Operating Characteristic” curve. 
from sklearn.metrics import roc_auc_score

roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

0.9564680456931152

In [47]:
table=pd.DataFrame.from_dict(result)

table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.151069,0.055730,0.022485,0.009330,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.852966,0.746799,0.644599,0.909115,0.872761,0.805248,0.096796,23
1,0.030930,0.006002,0.008121,0.004061,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.892857,0.857143,0.823129,0.874356,0.946153,0.878728,0.040797,1
2,0.032703,0.003939,0.006723,0.005524,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.833024,0.799537,0.731250,0.765625,0.890114,0.803910,0.054851,25
3,0.026979,0.007784,0.010382,0.000521,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.785714,0.658688,0.632414,0.753871,0.700900,0.706318,0.057140,40
4,0.039508,0.004779,0.010634,0.006464,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.852966,0.746799,0.644599,0.909115,0.872761,0.805248,0.096796,23
5,0.026975,0.004698,0.012322,0.003958,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.892857,0.857143,0.823129,0.874356,0.946153,0.878728,0.040797,1
6,0.037300,0.008219,0.008082,0.004041,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.833024,0.799537,0.731250,0.765625,0.890114,0.803910,0.054851,25
7,0.020743,0.000468,0.010320,0.000450,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.785714,0.714286,0.550097,0.753871,0.735075,0.707809,0.082279,39
8,0.166546,0.021990,0.010110,0.000023,100,auto,linear,"{'C': 100, 'gamma': 'auto', 'kernel': 'linear'}",0.852966,0.746799,0.644599,0.927778,0.872761,0.808981,0.100998,15
9,0.042867,0.006927,0.004344,0.004716,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",0.874254,0.859435,0.859435,0.874356,0.890114,0.871519,0.011431,3
